In [2]:
from glob import glob
from pprint import pprint
import textworld

In [3]:
filenames_train = glob("./games/train/*.json")
filenames_valid = glob("./games/valid/*.json")
filenames_test = glob("./games/test/*.json")
print(len(filenames_train))
print(len(filenames_valid))
print(len(filenames_test))

4440
222
514


In [62]:
# filter games (keep only one game per skillset)
names = {}
for filename in filenames_valid:
    prefix, seed = filename[:-5].rsplit("-", 1)
    names[prefix] = seed

filenames_valid = [name + "-" + seed + ".json" for name, seed in names.items()]
print(len(filenames_train))
print(len(filenames_valid))
print(len(filenames_test))

4440
222
466


In [18]:
for filename in filenames_valid:
    print(filename)

./games/valid/tw-cooking-recipe3+take3+open+drop-M3OYhOPxSWyJHyY1.json
./games/valid/tw-cooking-recipe3+cook+open+go12-7WDjCN08tjabF5Qg.json
./games/valid/tw-cooking-recipe1+cook+cut+open+drop+go6-rXmRCdv0Ia1YfpYG.json
./games/valid/tw-cooking-recipe1+take1+cook+cut+open+drop-p35nFk2NSJVqiV5L.json
./games/valid/tw-cooking-recipe3+take3+cut+go6-m67BIYnEUElZI3Ey.json
./games/valid/tw-cooking-recipe3+take3+cook+cut-bRoRtWeZho95hq8o.json
./games/valid/tw-cooking-recipe2+cut-pY7eFyRjs3lPhZb3.json
./games/valid/tw-cooking-recipe2+go12-v3gjIWOOUr9Rc5eE.json
./games/valid/tw-cooking-recipe1+take1+cut+open-2jOyfByjFOVnhZ89.json
./games/valid/tw-cooking-recipe2+cook+cut-P0J2c1D5UXeQF80W.json
./games/valid/tw-cooking-recipe2+cut+open+drop+go9-mNyji3a8tPLgFnaa.json
./games/valid/tw-cooking-recipe1+take1+open+drop-N0rRC7LgTmELf39g.json
./games/valid/tw-cooking-recipe3+take3+cut+open+drop-E6KPFabXUr3pU0va.json
./games/valid/tw-cooking-recipe3+go12-QP7RT6gDFWO7iO8m.json
./games/valid/tw-cooking-recip

In [ ]:
def load_games(filenames):
    return [textworld.Game.load(filename) for filename in filenames]

games_train = load_games(filenames_train)
games_valid = load_games(filenames_valid)
games_test = load_games(filenames_test)

#### Regenerate training games with drop skill.

In [4]:
def load_games(filenames):
    return [textworld.Game.load(filename) for filename in filenames if "drop" in filename]

games_train = load_games(filenames_train)
games_valid = load_games(filenames_valid)
games_test = load_games(filenames_test)

In [59]:
import tqdm
from subprocess import check_call

subset = "valid"

skipped = []
# Regenerate training games with drop skill.
for game in tqdm.tqdm(games_valid):
    if len(game.world.inventory.content) <= len(game.metadata["ingredients"]):
        skipped += [game.metadata["uuid"]]
        continue  # This game should be fine.
        
    challenge = game.metadata["uuid"].rsplit("-", 1)[0] + "+" + subset
    seeds = game.metadata["seeds"]
    cmd = ["tw-make", "challenge", challenge,
           "--seeds", seeds["map"], seeds["objects"], seeds["quest"], seeds["grammar"],
           "--output", "./games/" + subset + "2_drop/", "-f", "--silent"]
    check_call(map(str, cmd))
    new_game = textworld.Game.load("./games/" + subset + "2_drop/" + game.metadata["uuid"] + ".json")
    assert game.metadata["uuid"] == new_game.metadata["uuid"]
    assert game.metadata["seeds"] == new_game.metadata["seeds"]
    assert game.metadata["goal"] == new_game.metadata["goal"]
    assert game.metadata["ingredients"] == new_game.metadata["ingredients"]
    assert game.metadata["max_score"] == new_game.metadata["max_score"]
    #assert set(game.metadata["entities"]).issuperset(set(new_game.metadata["entities"]))









  0%|          | 0/90 [00:00<?, ?it/s]






  1%|          | 1/90 [00:04<06:02,  4.07s/it]






  6%|▌         | 5/90 [00:07<02:15,  1.60s/it]






  7%|▋         | 6/90 [00:12<02:56,  2.10s/it]






  8%|▊         | 7/90 [00:17<03:29,  2.52s/it]






 12%|█▏        | 11/90 [00:21<02:37,  1.99s/it]






 16%|█▌        | 14/90 [00:26<02:23,  1.88s/it]






 18%|█▊        | 16/90 [00:30<02:21,  1.91s/it]






 19%|█▉        | 17/90 [00:34<02:28,  2.04s/it]






 26%|██▌       | 23/90 [00:38<01:52,  1.68s/it]






 28%|██▊       | 25/90 [00:42<01:51,  1.71s/it]






 30%|███       | 27/90 [00:46<01:49,  1.73s/it]






 34%|███▍      | 31/90 [00:51<01:37,  1.65s/it]






 36%|███▌      | 32/90 [00:55<01:40,  1.73s/it]






 37%|███▋      | 33/90 [01:00<01:43,  1.82s/it]






 40%|████      | 36/90 [01:04<01:37,  1.80s/it]






 44%|████▍     | 40/90 [01:09<01:26,  1.73s/it]






 53%|█████▎    | 48/90 [01:13<01:04,  1.53s/it]






 56%|█████▌    | 50/90 [01:17<01:0

In [5]:
#items = [len(game.world.inventory.content) > len(game.metadata["ingredients"]) for game in games_train]
items = [(len(game.world.inventory.content), len(game.metadata["ingredients"]), game.metadata) for game in games_test]
for item in items:
    if item[0] > item[1] and not item[2]["skills"]["cut"]:
        pprint(item)

In [85]:
item

(5,
 4,
 {'seeds': {'map': 17161, 'objects': 20425, 'quest': 13318, 'grammar': 7312},
  'goal': "You open the copy of 'Cooking: A Modern Approach (3rd Ed.)' and start reading:\n\nRecipe #1\n---------\nGather all following ingredients and follow the directions to prepare this tasty meal.\n\nIngredients:\n  block of cheese\n  carrot\n  red onion\n  red potato\n\nDirections:\n  slice the block of cheese\n  fry the block of cheese\n  dice the carrot\n  fry the carrot\n  chop the red onion\n  roast the red potato\n  prepare meal\n",
  'ingredients': [['block of cheese', 'fried', 'sliced'],
   ['carrot', 'fried', 'diced'],
   ['red onion', 'raw', 'chopped'],
   ['red potato', 'roasted', 'uncut']],
  'skills': {'recipe': 4, 'take': 5, 'drop': True},
  'entities': ['meal',
   'kitchen',
   'table',
   'stove',
   'oven',
   'counter',
   'fridge',
   'cookbook',
   'carrot',
   'block of cheese',
   'red onion',
   'red potato',
   'red apple',
   'yellow onion',
   'red hot pepper',
   'yello

In [5]:
def get_ingredients(games):
    return [tuple(ingredient) for game in games for ingredient in game.metadata["ingredients"]]

ingredients_train = get_ingredients(games_train)
ingredients_valid = get_ingredients(games_valid)
ingredients_test = get_ingredients(games_test)

In [ ]:
from textworld.challenges.cooking import FOODS_SPLITS

def check_all_food_split(dataset, split):
    to_find = list(FOODS_SPLITS[split])
    print(to_find)
    for game in dataset:
        for food, cook, cut in game.metadata["ingredients"]:
            if food in to_find:
                to_find.remove(food)
                
    print(to_find)
    assert len(to_find) == 0
    
check_all_food_split(games_valid, "valid")
check_all_food_split(games_valid, "train")

In [15]:
from collections import defaultdict
from textworld.challenges.cooking import FOOD_PREPARATIONS_SPLITS

def count_food_preparation_in_split(dataset, split):
    to_find = [(food,) + preparation
               for food, preparations in FOOD_PREPARATIONS_SPLITS[split].items()
               for preparation in preparations]
    
    total = len(to_find)
    for game in dataset:
        for food, cook, cut in game.metadata["ingredients"]:
            preparation = (food, cook, cut)
            if preparation in to_find:
                to_find.remove(preparation)
    
    found = total - len(to_find)
    print("{} out of {} ({:.1%})".format(found, total, found/total))
    #print(to_find)

count_food_preparation_in_split(games_valid, "valid")
count_food_preparation_in_split(games_valid, "train")

        

56 out of 57 (98.2%)
5 out of 120 (4.2%)


In [ ]:
import textwrap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

def view_dataset(train, valid, test, titles=["train", "valid", "test"]):
    
    def build_formatter_fct():
        cpt = [0]
        def _fct(arg=None):
            if cpt[0] == 0:
                words = set(train) - set(valid) - set(test)
            elif cpt[0] == 1:
                words = set(valid) - set(train) - set(test)
            elif cpt[0] == 2:
                words = (set(train) & set(valid)) - set(test)
            elif cpt[0] == 3:
                words = set(test) - set(valid) - set(train)
            elif cpt[0] == 4:
                words = (set(test) & set(train)) - set(valid)
            elif cpt[0] == 5:
                words = (set(test) & set(valid)) - set(train)
            elif cpt[0] == 6:
                words = set(train) & set(valid) & set(test)
            else:
                return None

            cpt[0] += 1
            return "\n".join(sorted(words, key=lambda s: s.split()[::-1]))
            
        return _fct

    fct = build_formatter_fct()
    subsets = [fct() for _ in range(7)]
    sizes = [len(subset)**0.5 for subset in subsets]

    plt.figure(figsize=(16*1.5,9*1.5))
    venn = venn3(sizes, titles, subset_label_formatter=build_formatter_fct())
    plt.show()
    

In [ ]:
# Check food items splits of the datasets.
view_dataset(set(food for food, _, _ in ingredients_train),
             set(food for food, _, _ in ingredients_valid),
             set(food for food, _, _ in ingredients_test))

In [ ]:
# Check for missing food items in all datasets.
from textworld.challenges.cooking import FOODS
print(set(FOODS)
      - set(food for food, _, _ in ingredients_train)
      - set(food for food, _, _ in ingredients_valid)
      - set(food for food, _, _ in ingredients_test))

In [ ]:
titles = ("sliced", "diced", "chopped")
view_dataset([ingredient[0] for ingredient in ingredients_train if ingredient[2] == "sliced"],
             [ingredient[0] for ingredient in ingredients_train if ingredient[2] == "diced"],
             [ingredient[0] for ingredient in ingredients_train if ingredient[2] == "chopped"],
             titles)

view_dataset([ingredient[0] for ingredient in ingredients_valid if ingredient[2] == "sliced"],
             [ingredient[0] for ingredient in ingredients_valid if ingredient[2] == "diced"],
             [ingredient[0] for ingredient in ingredients_valid if ingredient[2] == "chopped"],
             titles)


view_dataset([ingredient[0] for ingredient in ingredients_test if ingredient[2] == "sliced"],
             [ingredient[0] for ingredient in ingredients_test if ingredient[2] == "diced"],
             [ingredient[0] for ingredient in ingredients_test if ingredient[2] == "chopped"],
             titles)

In [ ]:
import textwrap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

def view_dataset(train, valid, test=[]):
    
    def build_formatter_fct():
        cpt = [0]
        def _fct(arg=None):
            if cpt[0] == 0:
                words = set(train) - set(valid) - set(test)
            elif cpt[0] == 1:
                words = set(valid) - set(train) - set(test)
            elif cpt[0] == 2:
                words = (set(train) & set(valid)) - set(test)
            elif cpt[0] == 3:
                words = set(test) - set(valid) - set(train)
            elif cpt[0] == 4:
                words = (set(test) & set(train)) - set(valid)
            elif cpt[0] == 5:
                words = (set(test) & set(valid)) - set(train)
            elif cpt[0] == 6:
                words = set(train) & set(valid) & set(test)
            else:
                return None

            cpt[0] += 1
            return "\n".join(sorted(words, key=lambda s: s.split()[::-1]))
            
        return _fct

    fct = build_formatter_fct()
    subsets = [fct() for _ in range(7)]
    sizes = [len(subset)**0.5 for subset in subsets]

    plt.figure(figsize=(16*1.5,9*1.5))
    venn = venn3(sizes, ("train", "valid", "test"), subset_label_formatter=build_formatter_fct())
    plt.show()

foods_train = set(food for food, _, _ in ingredients_train)
foods_valid = set(food for food, _, _ in ingredients_valid)
common_food = foods_train & foods_valid

for food in common_food:
    view_dataset([ingredient for ingredient in ingredients_train if ingredient[0] == food],
                 [ingredient for ingredient in ingredients_valid if ingredient[0] == food])